In [ ]:
!git clone https://github.com/WiraDKP/neural_style_transfer.git

In [ ]:
cd neural_style_transfer

In [ ]:
import torch
from torch import optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

# Import Data

In [ ]:
from src.utils import load_image
from torchvision import transforms

In [ ]:
CONTENT_PATH = "data/content/najwa.jpg"
STYLE_PATH = "data/style/barli.jpg"

transform = transforms.Compose([
    transforms.Resize(300),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])  
    
content = load_image(CONTENT_PATH, transform).to(device)
style = load_image(STYLE_PATH, transform).to(device)

output = load_image(CONTENT_PATH, transform).to(device)
output.requires_grad = True

# Training Preparation -> MCO

In [ ]:
from src.model import NeuralStyleTransfer
from src.criterion import criterion

In [ ]:
model = NeuralStyleTransfer().to(device)
optimizer = optim.AdamW([output], lr=0.05)

# Training

## Extract Fitur

Layer yang direkomendasikan
```
["4", "8", "13", "20", "27"]
```

In [ ]:
content_features = model(content, layers=["4", "8"])
style_features = model(style, layers=["4", "8"])

## Training Loop

In [ ]:
from src.utils import draw_styled_image

In [ ]:
max_epochs = 2500
for epoch in range(1, max_epochs+1):
    output_features = model(output, layers=["4", "8"])
    loss = criterion(content_features, style_features, output_features, output_features, style_weight=1e6)
    loss.backward()
    
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % 100 == 0:
        print(f"Epoch: {epoch:5} | Loss: {loss.item():.5f}")
        _ = draw_styled_image(output)